In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [3]:
#Reading the data
df = pd.read_csv("../input/textrank/data.csv")
df.head

<bound method NDFrame.head of    article_id                                       article_text  source
0           1  Artificial intelligence (AI), sometimes called...     NaN>

In [4]:
df['article_text'][0]

'Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals. Colloquially, the term "artificial intelligence" is often used to describe machines (or computers) that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem solving".\n\nAs machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect. A quip in Tesler\'s Theorem says "AI is whatever hasn\'t been done yet." For instance, optical character recognition is frequently excluded from things considered to be AI, having become a routine technology. Modern machine capabilities ge

In [5]:
#splitting and tokenizing sentences
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

In [6]:
sentences[:5]

['Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals.',
 'Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.',
 'Colloquially, the term "artificial intelligence" is often used to describe machines (or computers) that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem solving".',
 'As machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect.',
 'A quip in Tesler\'s Theorem says "AI is whatever hasn\'t been done yet."']

In [7]:
# #Text preprocessing

clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
stop_words = stopwords.words('english')

def remove_stopwords(words: list) -> str:
    """Remove stopwords from a list of words and return the remaining words
       as a string joint by the space char. 
    """
    sentence = " ".join([word for word in words if word not in stop_words])
    return sentence

clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
print(sentences[:5])
clean_sentences[:5]

['Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals.', 'Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.', 'Colloquially, the term "artificial intelligence" is often used to describe machines (or computers) that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem solving".', 'As machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect.', 'A quip in Tesler\'s Theorem says "AI is whatever hasn\'t been done yet."']


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


['artificial intelligence ai sometimes called machine intelligence intelligence demonstrated machines unlike natural intelligence displayed humans animals',
 'leading ai textbooks define field study intelligent agents device perceives environment takes actions maximize chance successfully achieving goals',
 'colloquially term artificial intelligence often used describe machines computers mimic cognitive functions humans associate human mind learning problem solving',
 'machines become increasingly capable tasks considered require intelligence often removed definition ai phenomenon known ai effect',
 'quip tesler theorem says ai whatever done yet']

In [8]:
# Glove for word-word co-occurrence matrix
word_embeddings = {}
f = open('../input/glove6b/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [10]:
#Compute a vector for every sentence
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)


In [12]:
#finding cosine similarity between sentences
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [13]:
#converting the similarity matrix sim_mat into the graph,
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)


In [14]:
scores

{0: 0.04570723143620052,
 1: 0.0467804334452339,
 2: 0.04697500145623368,
 3: 0.04726095069058082,
 4: 0.03901166688436227,
 5: 0.04716362899149114,
 6: 0.04735448360695071,
 7: 0.045221424450518376,
 8: 0.046516030517214445,
 9: 0.046784300515000606,
 10: 0.04686259741026109,
 11: 0.047991079396709865,
 12: 0.04422225738351169,
 13: 0.04479541248442989,
 14: 0.046042008287349896,
 15: 0.04389767573868423,
 16: 0.04650309329586664,
 17: 0.04486126156273356,
 18: 0.0393655006333276,
 19: 0.04215950282176648,
 20: 0.04616660232166448,
 21: 0.04835785666990803}

In [15]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
print(len(ranked_sentences))
ranked_sentences[:5]

22


[(0.04835785666990803,
  'In the twenty-first century, AI techniques have experienced a resurgence following concurrent advances in computer power, large amounts of data, and theoretical understanding; and AI techniques have become an essential part of the technology industry, helping to solve many challenging problems in computer science, software engineering and operations research.'),
 (0.047991079396709865,
  'The traditional problems (or goals) of AI research include reasoning, knowledge representation, planning, learning, natural language processing, perception and the ability to move and manipulate objects.'),
 (0.04735448360695071,
  'Modern machine capabilities generally classified as AI include successfully understanding human speech, competing at the highest level in strategic game systems (such as chess and Go), autonomously operating cars, intelligent routing in content delivery networks, and military simulations.'),
 (0.04726095069058082,
  'As machines become increasingl

In [16]:
# ranked_sentences contains the highest ranked sentence for every text. It is a list of tuples (score, sentence)
# How can this be mapped back to the texts? Why do we need to compute the pagerank for every sentence against every other sentence?
for sentence in ranked_sentences:
    print(sentence[0], sentence[1])

0.04835785666990803 In the twenty-first century, AI techniques have experienced a resurgence following concurrent advances in computer power, large amounts of data, and theoretical understanding; and AI techniques have become an essential part of the technology industry, helping to solve many challenging problems in computer science, software engineering and operations research.
0.047991079396709865 The traditional problems (or goals) of AI research include reasoning, knowledge representation, planning, learning, natural language processing, perception and the ability to move and manipulate objects.
0.04735448360695071 Modern machine capabilities generally classified as AI include successfully understanding human speech, competing at the highest level in strategic game systems (such as chess and Go), autonomously operating cars, intelligent routing in content delivery networks, and military simulations.
0.04726095069058082 As machines become increasingly capable, tasks considered to re

In [17]:
#extract summaries from text based on their rankings for summary generation
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(1):
  print("ARTICLE:")
  print(df['article_text'][i])
  print('\n')
  print("SUMMARY:")
  print(ranked_sentences[i][1])
  print('\n')

ARTICLE:
Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals. Colloquially, the term "artificial intelligence" is often used to describe machines (or computers) that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem solving".

As machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect. A quip in Tesler's Theorem says "AI is whatever hasn't been done yet." For instance, optical character recognition is frequently excluded from things considered to be AI, having become a routine technology. Modern machine capabilitie